In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [152]:
import warnings
warnings.filterwarnings('ignore')

In [153]:
data = pd.read_csv('Churn_Modelling.csv')

In [154]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [155]:
data = data.iloc[:,3:]

In [156]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Encoding Categorical Data....


In [157]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [158]:
preprocessor = ColumnTransformer(
    transformers=[
        ('sc', StandardScaler(),['Age','Balance','EstimatedSalary']),
        ('ohe', OneHotEncoder(drop='first'), [1,2])
    ],
    remainder='passthrough'
    )

In [159]:
pipe = Pipeline(
    steps=[
        ('pipeline',preprocessor)
    ]
)

In [160]:
X = data.drop('Exited',axis=1)
y = data['Exited']

In [161]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [162]:
X_train.shape

(8000, 10)

In [163]:
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)

In [164]:
X_train[0]

array([-6.55785897e-01, -1.21847056e+00,  1.36766974e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  6.86000000e+02,  6.00000000e+00,
        2.00000000e+00,  1.00000000e+00,  1.00000000e+00])

In [165]:
import pickle
with open('pipe.pkl','wb') as f1:
    pickle.dump(pipe,f1)
    

In [166]:
### ANN Implementation...

In [167]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.models import Sequential
import datetime

In [168]:
model = Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## first layer with input interaction
        Dense(32,activation='relu'), ## HL2
        Dense(1,activation='sigmoid') # Output Layer
    ]
)

In [169]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                768       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [170]:
import tensorflow

In [171]:
### if we want our learning rate for different loss.
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()


In [172]:
### for fixed learning rate.
### Compiling the model.
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [173]:
model

In [174]:
### setting up the tensorboard.

### to create logs for visualisation.
log_dir = "logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [175]:
# set up early Stopping CallBack.
## to stop at some extent.
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [176]:
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 1s 3ms/step - loss: 0.9321 - accuracy: 0.7399 - val_loss: 0.6387 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.6903 - accuracy: 0.7449 - val_loss: 0.7596 - val_accuracy: 0.5535
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5279 - accuracy: 0.7730 - val_loss: 0.6684 - val_accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.6015 - accuracy: 0.7606 - val_loss: 0.4262 - val_accuracy: 0.8085
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.6020 - accuracy: 0.7576 - val_loss: 0.5369 - val_accuracy: 0.8025
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5906 - accuracy: 0.7703 - val_loss: 0.4524 - val_accuracy: 0.8075
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5332 - accuracy: 0.7786 - val_loss: 0.6436 - val_accuracy: 0.8030
Epoch 8/100

In [177]:
model.save('model2.h5')

In [178]:
#### Load TensorBoard Extensiion.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [179]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 24836), started 4:30:11 ago. (Use '!kill 24836' to kill it.)